# Callbacks

> Event handlers

In [ ]:
#| default_exp callbacks

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastai.callback.training import ShortEpochCallback, GradientAccumulation
from fastai.callback.tracker import  EarlyStoppingCallback, SaveModelCallback, ReduceLROnPlateau
from fastai.callback.schedule import ParamScheduler, SchedCos, SchedExp, SchedLin, SchedNo
from fastai.callback.core import Callback, range_of, Tensor
import matplotlib.pyplot as plt


## Training Callbacks

Callbacks that add functionlities during the training phase, including Callbacks that make decisions depending how a monitored metric/loss behaves

In [ ]:
#| export
class MeanLossGraphCallback(Callback):
    "Update a graph of training and validation loss"
    order,run_valid=65,False

    def before_fit(self):
        self.run = not hasattr(self.learn, 'lr_finder') and not hasattr(self, "gather_preds")
        if not(self.run): return
        self.nb_batches = []
        self.train_losses = []
        assert hasattr(self.learn, 'progress')

    def after_train(self): 
        self.nb_batches.append(self.train_iter)

    def after_epoch(self):
        "Plot validation loss in the pbar graph"
        if not self.nb_batches: return
        rec = self.learn.recorder
        epochs = range(len(self.nb_batches)) + 1
        self.train_losses.append(rec.log[1])
        val_losses = [v[1] for v in rec.values]
        x_bounds = (0, self.n_epoch)
        y_bounds = (0, max((max(Tensor(rec.losses)), max(Tensor(val_losses)))))
        self.progress.mbar.update_graph([(epochs, self.train_losses), (epochs, val_losses)], x_bounds, y_bounds)


In [ ]:
show_doc(ShortEpochCallback)

---

[source](https://github.com/fastai/fastai/blob/master/fastai/callback/training.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### ShortEpochCallback

>      ShortEpochCallback (pct=0.01, short_valid=True)

Fit just `pct` of an epoch, then stop

In [ ]:
show_doc(GradientAccumulation)

---

[source](https://github.com/fastai/fastai/blob/master/fastai/callback/training.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### GradientAccumulation

>      GradientAccumulation (n_acc=32)

Accumulate gradients before updating weights

In [ ]:
show_doc(EarlyStoppingCallback)

---

[source](https://github.com/fastai/fastai/blob/master/fastai/callback/tracker.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### EarlyStoppingCallback

>      EarlyStoppingCallback (monitor='valid_loss', comp=None, min_delta=0.0,
>                             patience=1, reset_on_fit=True)

A `TrackerCallback` that terminates training when monitored quantity stops improving.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| monitor | str | valid_loss | value (usually loss or metric) being monitored. |
| comp | NoneType | None | numpy comparison operator; np.less if monitor is loss, np.greater if monitor is metric. |
| min_delta | float | 0.0 | minimum delta between the last monitor value and the best monitor value. |
| patience | int | 1 | number of epochs to wait when training has not improved model. |
| reset_on_fit | bool | True | before model fitting, reset value being monitored to -infinity (if monitor is metric) or +infinity (if monitor is loss). |

In [ ]:
show_doc(SaveModelCallback)

---

[source](https://github.com/fastai/fastai/blob/master/fastai/callback/tracker.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### SaveModelCallback

>      SaveModelCallback (monitor='valid_loss', comp=None, min_delta=0.0,
>                         fname='model', every_epoch=False, at_end=False,
>                         with_opt=False, reset_on_fit=True)

A `TrackerCallback` that saves the model's best during training and loads it at the end.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| monitor | str | valid_loss | value (usually loss or metric) being monitored. |
| comp | NoneType | None | numpy comparison operator; np.less if monitor is loss, np.greater if monitor is metric. |
| min_delta | float | 0.0 | minimum delta between the last monitor value and the best monitor value. |
| fname | str | model | model name to be used when saving model. |
| every_epoch | bool | False | if true, save model after every epoch; else save only when model is better than existing best. |
| at_end | bool | False | if true, save model when training ends; else load best model if there is only one saved model. |
| with_opt | bool | False | if true, save optimizer state (if any available) when saving model. |
| reset_on_fit | bool | True | before model fitting, reset value being monitored to -infinity (if monitor is metric) or +infinity (if monitor is loss). |

In [ ]:
show_doc(ReduceLROnPlateau)

---

[source](https://github.com/fastai/fastai/blob/master/fastai/callback/tracker.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### ReduceLROnPlateau

>      ReduceLROnPlateau (monitor='valid_loss', comp=None, min_delta=0.0,
>                         patience=1, factor=10.0, min_lr=0, reset_on_fit=True)

A `TrackerCallback` that reduces learning rate when a metric has stopped improving.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| monitor | str | valid_loss | value (usually loss or metric) being monitored. |
| comp | NoneType | None | numpy comparison operator; np.less if monitor is loss, np.greater if monitor is metric. |
| min_delta | float | 0.0 | minimum delta between the last monitor value and the best monitor value. |
| patience | int | 1 | number of epochs to wait when training has not improved model. |
| factor | float | 10.0 | the denominator to divide the learning rate by, when reducing the learning rate. |
| min_lr | int | 0 | the minimum learning rate allowed; learning rate cannot be reduced below this minimum. |
| reset_on_fit | bool | True | before model fitting, reset value being monitored to -infinity (if monitor is metric) or +infinity (if monitor is loss). |

## Schedulers

Callback and helper functions to schedule hyper-parameters

In [ ]:
show_doc(ParamScheduler)

---

[source](https://github.com/fastai/fastai/blob/master/fastai/callback/schedule.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### ParamScheduler

>      ParamScheduler (scheds)

Schedule hyper-parameters according to `scheds`

`scheds` is a dictionary with one key for each hyper-parameter you want to schedule, with either a scheduler or a list of schedulers as values (in the second case, the list must have the same length as the the number of parameters groups of the optimizer).

In [ ]:
show_doc(SchedCos)

---

[source](https://github.com/fastai/fastai/blob/master/fastai/callback/schedule.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### SchedCos

>      SchedCos (start, end)

Cosine schedule function from `start` to `end`

In [ ]:
show_doc(SchedExp)

---

[source](https://github.com/fastai/fastai/blob/master/fastai/callback/schedule.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### SchedExp

>      SchedExp (start, end)

Exponential schedule function from `start` to `end`

In [ ]:
show_doc(SchedLin)

---

[source](https://github.com/fastai/fastai/blob/master/fastai/callback/schedule.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### SchedLin

>      SchedLin (start, end)

Linear schedule function from `start` to `end`

In [ ]:
show_doc(SchedNo)

---

[source](https://github.com/fastai/fastai/blob/master/fastai/callback/schedule.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### SchedNo

>      SchedNo (start, end)

Constant schedule function with `start` value

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()